# 그루핑

1. () : 패턴을 그룹화

2. group(인덱스)

3. 그루핑된 문자열 재참조하기

4. 그루핑된 문자열에 이름 붙이기

In [3]:
import re

str = "ABCABCABC"

p = re.compile(r"(ABC)+")
m = p.search(str)
m

<re.Match object; span=(0, 9), match='ABCABCABC'>

In [16]:
str = """이이름 010-1234-5678
김이름 010-1357-2468
Rachel 010-1000-1000"""

p = re.compile(r"^[a-zㄱ-ㅎ가-힣]+\s+010\D*\d{4}\D*\d{4}$", re.M | re.I)
m = p.findall(str)
m

['이이름 010-1234-5678', '김이름 010-1357-2468', 'Rachel 010-1000-1000']

In [18]:
p = re.compile(r"(^[a-zㄱ-ㅎ가-힣]+)\s+(010\D*\d{4}\D*\d{4}$)", re.M | re.I)

for m in p.finditer(str):
    print(f"m.group(): {m.group()}")
    print(f"m.group(1): {m.group(1)}")
    print(f"m.group(2): {m.group(2)}")

m.group(): 이이름 010-1234-5678
m.group(1): 이이름
m.group(2): 010-1234-5678
m.group(): 김이름 010-1357-2468
m.group(1): 김이름
m.group(2): 010-1357-2468
m.group(): Rachel 010-1000-1000
m.group(1): Rachel
m.group(2): 010-1000-1000


## 문자열 바꾸기 (치환)
1. sub
    - count: 지정하지 않으면 전체 문자열에서 모든 패턴을 치환, 숫자를 지정하면 지정된 숫자 만큼의 패턴만 치환
    - 첫 번째 매개변수에 함수가 올 수도 있다. 패턴 문자열만 가지고 치환하기 어려운 경우, 함수 내에 로직을 작성
2. subn
    - sub + count 기능 포함, 반환값은 튜플

In [21]:
str = """이이름 010-1234-5678
김이름 010-1357-2468
Rachel 010-1000-1000"""

p = re.compile(r"(^[a-zㄱ-ㅎ가-힣]+)\s+(010\D*\d{4}\D*\d{4}$)", re.M | re.I)

str2 = p.sub(r"\g<2> \g<1>", str)
print(str2)

010-1234-5678 이이름
010-1357-2468 김이름
010-1000-1000 Rachel


- 그룹핑의 이름을 설정
```python
(?P<이름>)
```

In [22]:
str = """이이름 010-1234-5678
김이름 010-1357-2468
Rachel 010-1000-1000"""

p = re.compile(r"(?P<name>^[a-zㄱ-ㅎ가-힣]+)\s+(?P<mobile>010\D*\d{4}\D*\d{4}$)", re.M | re.I)

str2 = p.sub(r"\g<2> \g<1>", str)
print(str2)

010-1234-5678 이이름
010-1357-2468 김이름
010-1000-1000 Rachel


In [ ]:
- 그룹화된 패턴을 재활용

In [27]:
str = "Paris in the the spring"  # the를 2번 반복한 오타를 찾고 싶다

# p = re.compile(r"(\w+)\s+\1")  # \1: 앞의 1번 그룹과 같은 문자열과 매치
p = re.compile(r"(?P<word>\w+)\s+(?P=word)")

m = p.search(str)
m

<re.Match object; span=(9, 16), match='the the'>

In [30]:
str = "one little, two little, three little indians"

p = re.compile("little")
# str2 = p.sub("big", str)
str2 = p.sub("big", str, count=2)  # 왼쪽부터 매칭되는 패턴 2개만 치환
str2

'one big, two big, three little indians'

In [29]:
result = p.subn("big", str, 2)
result

('one big, two big, three little indians', 2)

In [31]:
str2 = p.sub(lambda m: m.group().upper(), str)
str2

'one LITTLE, two LITTLE, three LITTLE indians'

## 전방 탐색

1. 긍정형 전방 탐색
- 특정 패턴의 앞쪽 패턴으로 한정\
`앞쪽패턴(?=특정패턴)`
2. 부정형 전방 탐색
- 특정 패턴이 아닌 앞쪽 패턴으로 한정\
`앞쪽패턴(?!특정패턴)`

In [47]:
str = """https://www.naver.com
https://www.daum.net
http://www.webnmobile.net"""

# http, https를 추출하고 싶은데
# 주소 내에 포함되어 있어도 인식한다.
# -> :// 앞쪽에 있는 http, https만 가져오자

# p = re.compile(r"http[s]?://", re.M)
p = re.compile(r"http[s]?(?=://)", re.M)

items = p.findall(str)
items

['https', 'https', 'http']

In [38]:
p = re.compile(r"http[s]?://(www\.)?([^.]+(?=\.net$))", re.M)

for item in p.finditer(str):
    print(item.group(2))

daum
webnmobile


In [48]:
p = re.compile(r"http[s]?://(www\.)?(.+(?!\.net$))", re.M) # 전방 부정 탐색, .net으로 끝나지 않는 앞쪽 패턴 문자

print(str)
for item in p.finditer(str):
    print(item.group(2))

https://www.naver.com
https://www.daum.net
http://www.webnmobile.net
naver.com
daum.net
webnmobile.net


### greedy와 non-greedy

- 패턴+, 패턴*, 패턴?, 패턴{n,m} : 최대 매칭(greedy)
- 패턴+?, 패턴*?, 패턴??, 패턴{n,m}? : 최소 매칭(non-greedy)

In [43]:
str = "<html><head><title>사이트제목</title></head><body></body></html>"

p = re.compile(r"<.*>")

m = p.search(str)
m.group()

'<html><head><title>사이트제목</title></head><body></body></html>'

In [44]:
p = re.compile(r"<.*?>")  # non-greedy: 최소 매핑
m = p.search(str)
m.group()

'<html>'